In [95]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import os
os.chdir(os.path.expanduser('~/dev/vEcoli')) #import repo

In [116]:
sheets= pd.read_excel('notebooks/Heena notebooks/Mia Protein Degradation/41467_2024_49920_MOESM4_ESM.xlsx').iloc[3:, 1:].reset_index(drop=True)

sheets.columns= sheets.iloc[0]
sheets= sheets[1:]

col_of_interest = ['Protein ID', 'Gene names ', 'C-lim3_1', 'C-lim3_2', 'C-lim3_mean', 'C-lim6_1',
       'C-lim6_2', 'C-lim6_mean', 'C-lim12_1', 'C-lim12_2', 'C-lim12_mean']
df_clim = sheets[col_of_interest].copy()
df_clim

,Protein ID,Gene names,C-lim3_1,C-lim3_2,C-lim3_mean,C-lim6_1,C-lim6_2,C-lim6_mean,C-lim12_1,C-lim12_2,C-lim12_mean
1,sp|A5A614|YCIZ_ECOLI,yciZ,NaN,NaN,NaN,3.932642,3.601144,3.766893,NaN,NaN,NaN
2,sp|O32583|THIS_ECOLI,thiS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.469682,4.469682
3,sp|P00350|6PGD_ECOLI,gnd,3.248315,3.163151,3.205733,5.572427,6.131487,5.851957,10.201045,9.866244,10.033644
4,sp|P00363|FRDA_ECOLI,frdA,2.992966,2.946457,2.969711,6.080384,6.670409,6.375396,13.518765,10.761427,12.140096
5,sp|P00370|DHE4_ECOLI,gdhA,3.076857,2.810401,2.943629,4.462108,4.855534,4.658821,7.653326,6.91881,7.286068
...,...,...,...,...,...,...,...,...,...,...,...
3258,sp|P0AAD4|TYRP_ECOLI,tyrP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259,sp|P42592|YGJK_ECOLI,ygjK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3260,sp|P69330|CITD_ECOLI,citD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3261,sp|P77294|YDER_ECOLI,ydeR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# data cleaning
# (1) convert half life columns to numeric if there is a '*' in the value
half_life_cols = col_of_interest[2:]
for col in half_life_cols:
    df_clim[col] = (df_clim[col].astype(str)
                    .str.replace(r'[\s\*]+$', '', regex=True)   # drop trailing * or spaces
                    .pipe(pd.to_numeric, errors='coerce')
                    )
df_clim_uncapped = df_clim.copy()

# (1) ceil mean half life at dilution time for each column
dilution_times= {
                 'C-lim3':3, # hr
                 'C-lim6':6, # hr
                 'C-lim12':12 # hr
                }
col_names = df_clim.columns

for condition in dilution_times.keys():
    rep_col1 = f"{condition}_mean"

    dilution_time = dilution_times[condition]
    df_clim[rep_col1] = df_clim[rep_col1].apply(lambda x: min(x, dilution_time))

# (2) calculate stdev of each condition across the two replicates
for condition in dilution_times.keys():
    rep1_col = f"{condition}_1"
    rep2_col = f"{condition}_2"
    var_col = f"{condition}_var"
    df_clim[var_col] = np.std(df_clim[[rep1_col, rep2_col]], axis=1)
    df_clim_uncapped[var_col] = np.std(df_clim_uncapped[[rep1_col, rep2_col]], axis=1)
df_clim
df_clim_uncapped

,Protein ID,Gene names,C-lim3_1,C-lim3_2,C-lim3_mean,C-lim6_1,C-lim6_2,C-lim6_mean,C-lim12_1,C-lim12_2,C-lim12_mean,C-lim3_var,C-lim6_var,C-lim12_var
1,sp|A5A614|YCIZ_ECOLI,yciZ,NaN,NaN,NaN,3.932642,3.601144,3.766893,NaN,NaN,NaN,NaN,0.165749,NaN
2,sp|O32583|THIS_ECOLI,thiS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.469682,4.469682,NaN,NaN,0.000000
3,sp|P00350|6PGD_ECOLI,gnd,3.248315,3.163151,3.205733,5.572427,6.131487,5.851957,10.201045,9.866244,10.033644,0.042582,0.279530,0.167401
4,sp|P00363|FRDA_ECOLI,frdA,2.992966,2.946457,2.969711,6.080384,6.670409,6.375396,13.518765,10.761427,12.140096,0.023255,0.295012,1.378669
5,sp|P00370|DHE4_ECOLI,gdhA,3.076857,2.810401,2.943629,4.462108,4.855534,4.658821,7.653326,6.918810,7.286068,0.133228,0.196713,0.367258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,sp|P0AAD4|TYRP_ECOLI,tyrP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3259,sp|P42592|YGJK_ECOLI,ygjK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3260,sp|P69330|CITD_ECOLI,citD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3261,sp|P77294|YDER_ECOLI,ydeR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
# Calculate degradation rates based on constant model
def get_active_deg_model(tau_1, tau_2, dil_time1, dil_time2):
    num = np.log(2) * (tau_2/dil_time2 - tau_1/dil_time1)
    denom = tau_1 - tau_2
    k_act = num/denom
    return k_act

def get_active_deg_linear(tau_1, dil_time1):
    k_act = np.log(2) / tau_1 - np.log(2) / dil_time1
    return k_act

def get_df_active_deg(df_clims):
    df_active_deg = pd.DataFrame()
    pair_wise_comparison = {
                            '3_vs_12': ('C-lim3', 'C-lim12'),
                            '6_vs_3': ('C-lim6', 'C-lim3'),
                            '12_vs_6': ('C-lim12', 'C-lim6')
                           }
    col_names_ratio = []
    col_names_linear = []

    for comparison, (condition_1, condition_2) in pair_wise_comparison.items():
        rep_col1 = f"{condition_1}_mean"
        rep_col2 = f"{condition_2}_mean"

        dil_time1 = dilution_times[condition_1]
        dil_time2 = dilution_times[condition_2]
        tau_1 = df_clims[rep_col1]
        tau_2 = df_clims[rep_col2]
        k_act_col = f"k_active_{condition_1}_{condition_2}"

        col_ratio = f"{k_act_col}_ratio"
        col_linear = f"k_active_{condition_1}_linear"
        col_names_ratio.append(col_ratio)
        col_names_linear.append(col_linear)

        df_active_deg[col_ratio] = get_active_deg_model(tau_1, tau_2, dil_time1, dil_time2)
        df_active_deg[col_linear] = get_active_deg_linear(tau_1, dil_time1)

    df_active_deg['standard_deviation_ratio'] = df_active_deg[col_names_ratio].std(axis=1)
    df_active_deg['mean_ratio'] = df_active_deg[col_names_ratio].mean(axis=1)
    df_active_deg['standard_deviation_linear'] = df_active_deg[col_names_linear].std(axis=1)
    df_active_deg['mean_linear'] = df_active_deg[col_names_linear].mean(axis=1)
    df_active_deg.index = df_clims['Gene names ']
    return df_active_deg

In [119]:
df_active_deg = get_df_active_deg(df_clim)
df_active_deg_uncapped = get_df_active_deg(df_clim_uncapped)

# Plot 1: Scatterplot

In [44]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Map each condition to (ratio_x, linear_y) columns
COLMAP = {
    "C-lim3":  ("k_active_C-lim3_C-lim12_ratio",  "k_active_C-lim3_linear"),
    "C-lim6":  ("k_active_C-lim6_C-lim3_ratio", "k_active_C-lim6_linear"),
    "C-lim12": ("k_active_C-lim12_C-lim6_ratio", "k_active_C-lim12_linear"),
}

def plotly_scatter(df, condition="C-lim3", log_scale=True):
    xcol, ycol = COLMAP[condition]
    d = df[[xcol, ycol]].copy()

    # keep finite & positive (needed for log scale)
    d = d.replace([np.inf, -np.inf], np.nan).dropna()

    # axis span for identity line
    lo = float(np.minimum(d[xcol].min(), d[ycol].min()))
    hi = float(np.maximum(d[xcol].max(), d[ycol].max()))
    print(lo,hi)
    fig = go.Figure()

    fig.add_scatter(
        x=d[xcol], y=d[ycol],
        mode="markers", name="points",
        marker=dict(size=6, opacity=0.7),
        hovertemplate=f"{xcol}: %{{x:.3g}}<br>{ycol}: %{{y:.3g}}<extra></extra>"
    )

    # y = x identity line
    fig.add_scatter(
        x=[lo, hi], y=[lo, hi],
        mode="lines", name="y = x",
        line=dict(dash="dash"),
        hoverinfo="skip"
    )

    fig.update_layout(
        title=f"Linear vs Ratio — {condition}",
        xaxis_title=f"Ratio ({xcol})",
        yaxis_title=f"Linear ({ycol})",
        template="plotly_white",
        margin=dict(l=60, r=20, t=60, b=60),
        showlegend=False
    )

    if log_scale:
        fig.update_layout(
            title=f"Linear vs Ratio — {condition} - Log-Log Scale")
        fig.update_xaxes(type="log")
        fig.update_yaxes(type="log")
    return fig

In [65]:
from plotly.subplots import make_subplots

conditions = ["C-lim3", "C-lim6", "C-lim12"]  # 3 rows
titles = np.array([[f"{c} — linear", f"{c} — log"] for c in conditions])
titles = titles.reshape(6)

SCATTER_COLOR = "rgba(255, 99, 132, 0.8)"   # points
LINE_COLOR    = "rgba(60, 60, 60, 1.0)"     # y = x lines


fig = make_subplots(
    rows=len(conditions), cols=2,
    subplot_titles=titles,
    horizontal_spacing=0.08, vertical_spacing=0.12
)

for r, cond in enumerate(conditions, start=1):
    # left column: linear
    fig_linear = plotly_scatter(df_active_deg, condition=cond, log_scale=False)
    for tr in fig_linear.data:
        tr.showlegend = False  # avoid repeated legends
        fig.add_trace(tr, row=r, col=1)
    # if your helper titles axes, bring those over:
    fig.update_xaxes(title_text=fig_linear.layout.xaxis.title.text, row=r, col=1)
    fig.update_yaxes(title_text=fig_linear.layout.yaxis.title.text, row=r, col=1)

    # right column: log
    fig_log = plotly_scatter(df_active_deg, condition=cond, log_scale=True)
    for tr in fig_log.data:
        tr.showlegend = False
        fig.add_trace(tr, row=r, col=2)
    # enforce log axes for that cell
    fig.update_xaxes(type="log", title_text=fig_log.layout.xaxis.title.text, row=r, col=2)
    fig.update_yaxes(type="log", title_text=fig_log.layout.yaxis.title.text, row=r, col=2)

fig.update_layout(
    height=300*len(conditions), width=950,
    # plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot area background
    # paper_bgcolor='rgba(0, 0, 0, 0)',
    title_text="Method-vs-Method: linear vs log by condition",
    template="plotly_white",
    legend_tracegroupgap=8
)

# unify colors
fig.update_traces(marker=dict(color=SCATTER_COLOR),
                  selector=dict(mode="markers"))
fig.update_traces(line=dict(color=LINE_COLOR, dash="dash", width=2),
                  selector=dict(mode="lines"))

fig.show()
# fig.write_image("notebooks/Heena notebooks/Mia Protein Degradation/out/k_active_method_comparison.png", scale=5, height=900, width=950)

-166.9954918933256 286961618.2365557
-166.9954918933256 286961618.2365557
-45.35968073426505 177.5155619017341
-45.35968073426505 177.5155619017341
-16.92172371558435 13.433382554916355
-16.92172371558435 13.433382554916355


#### Plot 1 Comment: Plotting in log-scale can better highlight how the linear and ratio method match, so I will continue the analysis in log-scale plots. The ratio method produces more negative values while the linear model does not (sicne we have done data cleaning to cap half-time at dilution time). For negative proteins with negative ratio method, we can supplement with linear method.

# Plot 2: Scatterplot showing the agreement of k_active obtained from each method (pool condition).

In [54]:
METHOD_COLS = {
    "linear": {
        "C-lim3":  "k_active_C-lim3_linear",
        "C-lim6":  "k_active_C-lim6_linear",
        "C-lim12": "k_active_C-lim12_linear",
    },
    "ratio": {
        # These are the "per-condition" columns you used earlier
        "C-lim3":  "k_active_C-lim3_C-lim12_ratio",
        "C-lim6":  "k_active_C-lim6_C-lim3_ratio",
        "C-lim12": "k_active_C-lim12_C-lim6_ratio",
    },
}

def scatter_condition_agreement(df, method="linear", log_axes=True,
                                point_color="rgba(255,99,132,0.8)",
                                line_color="rgba(60,60,60,1.0)"):
    cols = METHOD_COLS[method]
    pairs = [("C-lim3","C-lim6"), ("C-lim6","C-lim12"), ("C-lim3","C-lim12")]

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=[f"{a} vs {b}" for a,b in pairs],
        horizontal_spacing=0.08
    )

    for j, (a, b) in enumerate(pairs, start=1):
        x = df[cols[a]].replace([np.inf, -np.inf], np.nan)
        y = df[cols[b]].replace([np.inf, -np.inf], np.nan)

        m = x.notna() & y.notna()
        x = x[m].to_numpy(float)
        y = y[m].to_numpy(float)

        if log_axes:
            mpos = (x > 0) & (y > 0)
            x = x[mpos]; y = y[mpos]

        if len(x) == 0:
            # add an empty trace so axes still render
            fig.add_trace(go.Scatter(x=[], y=[], mode="markers"), row=1, col=j)
            continue

        # identity line limits
        lo = float(min(x.min(), y.min()))
        hi = float(max(x.max(), y.max()))

        fig.add_trace(
            go.Scatter(
                x=x, y=y, mode="markers",
                marker=dict(color=point_color, size=6, opacity=0.7),
                name="proteins",
                hovertemplate=(f"{cols[a]}: %{{x:.4g}}<br>"
                               f"{cols[b]}: %{{y:.4g}}<extra></extra>")
            ),
            row=1, col=j
        )

        fig.add_trace(
            go.Scatter(
                x=[lo, hi], y=[lo, hi], mode="lines",
                line=dict(color=line_color, dash="dash", width=2),
                name="y = x", hoverinfo="skip", showlegend=False
            ),
            row=1, col=j
        )

        fig.update_xaxes(title_text=f"{a}", row=1, col=j,
                         type="log" if log_axes else None)
        fig.update_yaxes(title_text=f"{b}", row=1, col=j,
                         type="log" if log_axes else None)

    fig.update_layout(
        title=f"Condition agreement within method: {method}",
        template="plotly_white",
        height=380, width=1050,
        showlegend=False
    )
    return fig

In [73]:
fig = scatter_condition_agreement(df_active_deg, method="linear", log_axes=True)
# cap x and y-axis
fig.update_layout(yaxis1=dict(range=[-5, 2]),
                  yaxis2=dict(range=[-5, 2]),
                  yaxis3=dict(range=[-5, 2]),
                  xaxis1=dict(range=[-5, 2]),
                  xaxis2=dict(range=[-5, 2]),
                  xaxis3=dict(range=[-5, 2]),
                  # plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot area background
                  # paper_bgcolor='rgba(0, 0, 0, 0)'
                  )
fig.show()
# fig.write_image("notebooks/Heena notebooks/Mia Protein Degradation/out/k_active_condition_comparison_linear.png", scale=5, height=400, width=1000)

In [75]:
fig = scatter_condition_agreement(df_active_deg, method="ratio", log_axes=True)
fig.update_layout(yaxis1=dict(range=[-5, 2]),
                  yaxis2=dict(range=[-5, 2]),
                  yaxis3=dict(range=[-5, 2]),
                  xaxis1=dict(range=[-5, 2]),
                  xaxis2=dict(range=[-5, 2]),
                  xaxis3=dict(range=[-5, 2]),
                  # plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot area background
                  # paper_bgcolor='rgba(0, 0, 0, 0)'
                  )
fig.show()
# fig.write_image("notebooks/Heena notebooks/Mia Protein Degradation/out/k_active_condition_comparison_log.png", scale=5, height=400, width=1000)

In [78]:
import numpy as np
import pandas as pd

# --- Your columns per method (adjust if needed) ---
METHOD_COLS = {
    "linear": {
        "C-lim3":  "k_active_C-lim3_linear",
        "C-lim6":  "k_active_C-lim6_linear",
        "C-lim12": "k_active_C-lim12_linear",
    },
    "ratio": {
        "C-lim3":  "k_active_C-lim3_C-lim12_ratio",
        "C-lim6":  "k_active_C-lim6_C-lim3_ratio",
        "C-lim12": "k_active_C-lim12_C-lim6_ratio",
    },
}

def _signed_log10(a):
    """Signed-log transform that can handle <=0: s(x)=sign(x)*log10(1+|x|)."""
    a = np.asarray(a, dtype=float)
    return np.sign(a) * np.log10(1.0 + np.abs(a))

def agreement_scores_3d(df: pd.DataFrame,
                        method_cols: dict,
                        keep_nonpositive=False):
    """
    Compute (1) RMS perpendicular distance to the diagonal and
    (3) R^2_diag for each method, using three condition columns.

    Parameters
    ----------
    df : DataFrame with the k_active columns.
    method_cols : dict like {"linear": {"C-lim3": col, "C-lim6": col, "C-lim12": col}, ...}
    keep_nonpositive : if False (default) drop rows with any <=0 before log10;
                       if True, use signed-log so rows are kept.

    Returns
    -------
    scores : DataFrame with rows per method and columns:
             ['n', 'rms_perp_log10', 'R2_diag', 'note']
    details : dict of per-method arrays (optional debugging)
    """
    u = np.ones(3) / np.sqrt(3.0)  # unit vector along the diagonal (1,1,1)
    rows = []
    details = {}

    for method, cols in method_cols.items():
        X = df[[cols["C-lim3"], cols["C-lim6"], cols["C-lim12"]]].to_numpy(float)

        if keep_nonpositive:
            # signed-log keeps all rows
            Xlog = _signed_log10(X)
            note = "signed-log10(1+|x|)"
        else:
            # drop rows with any nonpositive or NaN/Inf, then log10
            mask = np.all(np.isfinite(X), axis=1) & np.all(X > 0, axis=1)
            X = X[mask]
            Xlog = np.log10(X)
            note = "log10, positives only"

        n = Xlog.shape[0]
        if n == 0:
            rows.append((method, 0, np.nan, np.nan, note))
            details[method] = {"Xlog": Xlog, "mask_kept": []}
            continue

        # center for total variance
        mu = Xlog.mean(axis=0, keepdims=True)

        # projection onto diagonal and perpendicular residuals
        # proj = (Xlog·u) u ; perp = Xlog - proj
        dot = Xlog @ u
        proj = np.outer(dot, u)
        perp = Xlog - proj

        # (1) RMS perpendicular distance in log10 units
        d_perp = np.linalg.norm(perp, axis=1)
        rms_perp = np.sqrt(np.mean(d_perp ** 2))

        # (3) R^2 along diagonal
        V_perp = np.mean(np.sum(perp ** 2, axis=1))
        V_tot  = np.mean(np.sum((Xlog - mu) ** 2, axis=1))
        R2_diag = np.nan if V_tot == 0 else 1.0 - (V_perp / V_tot)

        rows.append((method, n, rms_perp, R2_diag, note))
        details[method] = {
            "Xlog": Xlog, "dot_along_diag": dot, "perp_dist": d_perp,
            "V_perp": V_perp, "V_tot": V_tot
        }

    scores = pd.DataFrame(rows, columns=["method", "n", "rms_perp_log10", "R2_diag", "note"])
    return scores.set_index("method"), details


In [85]:
scores, details = agreement_scores_3d(df_active_deg, METHOD_COLS, keep_nonpositive=False)
print(scores)

           n  rms_perp_log10   R2_diag                   note
method                                                       
linear   982        0.504516  0.818124  log10, positives only
ratio   1138        0.520102  0.711458  log10, positives only


# Plot 3: Heatmap showing k_active distribution per method

In [136]:
import plotly.express as px
# --- adjust if your column names differ ---
METHOD_COLS = [
        "k_active_C-lim3_linear",
        "k_active_C-lim6_linear",
        "k_active_C-lim12_linear",
        "k_active_C-lim3_C-lim12_ratio",
        "k_active_C-lim6_C-lim3_ratio",
        "k_active_C-lim12_C-lim6_ratio"]

def half_life_heatmap(df, method_cols=METHOD_COLS, k_units="per_hour"):
    """
    Build a 2 x 6 heatmap of counts: rows=methods, cols=half-life bins.

    Parameters
    ----------
    df : DataFrame with k_active columns
    method_cols : dict -> method name -> list of columns for that method
    k_units : "per_hour" or "per_second" (set to per_second if k is s^-1)

    Notes
    -----
    - Negative k -> negative half-life -> counted in '<0'
    - k == 0 or non-finite -> ignored (not counted in any bin)
    - Positive k are binned by t1/2 in hours into: 0–1, 1–2, 2–4, 4–8, 8–12
    """
    # convert ln(2)/k to hours
    LN2 = np.log(2.0)
    sec_per_hour = 3600.0
    scale = 1.0 if k_units == "per_hour" else (sec_per_hour if k_units == "per_second" else 1.0)

    # bin edges in hours (include -inf for negatives)
    edges = np.array([-np.inf, 0, 1, 2, 4, 8, 12, np.inf], dtype=float)
    labels = ["<0", "0–1 h", "1–2 h", "2–4 h", "4–8 h", "8–12 h", ">12h"]

    # container for counts per method
    z = []
    row_names = []

    for cols in method_cols:
        # collect all k values for this method (flatten across its columns)
        K = df[cols].to_numpy(dtype=float).ravel()

        # drop nans/infs up front for safer arithmetic
        finite_mask = np.isfinite(K)
        K = K[finite_mask]

        # separate negatives (go straight to "<0")
        neg_mask = K < 0
        n_neg = int(neg_mask.sum())

        # positives -> compute half-life in hours
        pos = K[~neg_mask]
        # remove zeros to avoid inf half-life
        pos = pos[pos > 0]
        t_half_hours = (LN2 / pos) * (1.0 if k_units == "per_hour" else (1.0 / sec_per_hour if k_units == "per_second" else 1.0))
        # If your rates are s^-1, set k_units="per_second" so the conversion above yields hours.

        # histogram for bins 0–1,1–2,2–4,4–8,8–12 (note: negatives already handled separately)
        # We'll histogram over full edges and then replace the first bin count with n_neg
        counts, _ = np.histogram(t_half_hours, bins=edges)
        counts = counts.astype(int)
        counts[0] = n_neg  # first column "<0"

        z.append(counts.tolist())
        row_names.append(cols)

    # build a tidy DataFrame for clarity (optional)
    heat_df = pd.DataFrame(z, index=row_names, columns=labels)

    # plot
    fig = px.imshow(
        heat_df.values,
        x=labels, y=row_names,
        text_auto=True, color_continuous_scale="Blues",
        labels=dict(x="Half-life bin", y="Method", color="Count"),
        aspect="auto"
    )
    fig.update_layout(title="Half-life distribution (counts) by method", template="plotly_white")
    fig.update_xaxes(side="top")
    return fig, heat_df


In [139]:
fig, heat_df = half_life_heatmap(df_active_deg_uncapped, METHOD_COLS, k_units="per_hour")
fig.show()
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot area background
    paper_bgcolor='rgba(0, 0, 0, 0)'
)
# fig.write_image("notebooks/Heena notebooks/Mia Protein Degradation/out/half_life_distribution.png", scale=5, height=400, width=1000)
heat_df['total_protein'] = heat_df.sum(axis=1)
print(heat_df)

                                 <0  0–1 h  1–2 h  2–4 h  4–8 h  8–12 h  >12h  \
k_active_C-lim3_linear         1333     66     71    105    129     111   850   
k_active_C-lim6_linear          769     38     48     94    137     114  1451   
k_active_C-lim12_linear         508     27     43     96    140     126  1757   
k_active_C-lim3_C-lim12_ratio   366     21     20     68    148     127  1777   
k_active_C-lim6_C-lim3_ratio    640     31     44     71    237     248  1242   
k_active_C-lim12_C-lim6_ratio   641     17     37     61    157     160  1473   

                               total_protein  
k_active_C-lim3_linear                  2665  
k_active_C-lim6_linear                  2651  
k_active_C-lim12_linear                 2697  
k_active_C-lim3_C-lim12_ratio           2527  
k_active_C-lim6_C-lim3_ratio            2513  
k_active_C-lim12_C-lim6_ratio           2546  
